In [ ]:
# nse_main.py
# Main program for nse

from ib_insync import *
util.startLoop()

from helper import *
from nse_func import *

if __name__ == '__main__':
    with get_connected('nse', 'live') as ib:
        nse_weekend_process(ib) # generates lot margin and option pickles
    with get_connected('nse', 'live') as ib:
        nse_everyday_process(ib) # generates target list after evlaluation

In [ ]:
from ib_insync import *
util.startLoop()

from helper import *
from nse_func import *
ib =  get_connected('nse', 'live')

# ...update underlyings
dfu = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

In [ ]:
# ...update option prices
fs = listdir(fspath)

# collect the options
optsList = [f for f in fs if f[-3:] == 'pkl']
dfopts = pd.concat([pd.read_pickle(fspath+f)
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)

In [ ]:
dfopts